In [3]:
import pandas as pd
import os
import pandas as pd
from allennlp_models import pretrained
from group9_SRL import predict_total
import re
import string as string_value
# from predict import predict_SRL, predict_SRL_BERT, predict_srl_group9
import predict as pred
sentence = "I am a student"
allen_models = ["structured-prediction-srl-bert", 'structured-prediction-srl']

MODEL_SRL_BERT = pretrained.load_predictor(allen_models[0])
MODEL_SRL = pretrained.load_predictor(allen_models[1])

error loading _jsonnet (this is expected on Windows), treating C:\Users\jonas\anaconda3\Lib\site-packages\allennlp_models\modelcards\coref-spanbert.json as plain json
error loading _jsonnet (this is expected on Windows), treating C:\Users\jonas\anaconda3\Lib\site-packages\allennlp_models\modelcards\evaluate_rc-lerc.json as plain json
lerc is not a registered model.
error loading _jsonnet (this is expected on Windows), treating C:\Users\jonas\anaconda3\Lib\site-packages\allennlp_models\modelcards\generation-bart.json as plain json
error loading _jsonnet (this is expected on Windows), treating C:\Users\jonas\anaconda3\Lib\site-packages\allennlp_models\modelcards\glove-sst.json as plain json
error loading _jsonnet (this is expected on Windows), treating C:\Users\jonas\anaconda3\Lib\site-packages\allennlp_models\modelcards\lm-masked-language-model.json as plain json
error loading _jsonnet (this is expected on Windows), treating C:\Users\jonas\anaconda3\Lib\site-packages\allennlp_models\mod

In [6]:
import os
import pandas as pd
from allennlp_models import pretrained
from group9_SRL import predict_total
import re
import string as string_value



def get_constituent(predicate, srl_output):
    
    # get the description of the srl out_put corresponding to the predicate

    for verb in srl_output['verbs']:
        if verb['verb'] == predicate:
            description = verb['description']
            return get_constituent_list(description)
    
            


def get_constituent_list(string):

    # define a regular expression pattern to match the relevant words
    pattern = r'\[(ARG\d+|V):\s*([^\]]+)\]|(\b\w+\b)'

    # find all matches of the pattern in the string
    matches = re.findall(pattern, string)

    # extract the relevant words from the matches and store them in a list
    words = [match[1] or match[2] for match in matches]

    # append the final period to the list
    if string[-1] in string_value.punctuation:

        words.append(string[-1])

    return words    


def parse_string(input_string):
    """
    Parses the string output of the SRL model into a dictionary
    """
    # define a regular expression pattern to match the string format
    pattern = r"\[(\w+): ([^]]+)\]"
        # search for matches using the pattern
    matches = re.findall(pattern, input_string)
    
    # convert the matches into a dictionary
    output_dict = {}
    for match in matches:
        key = match[0]
        value = match[1]
        output_dict[key] = value
    
    return output_dict

def classify(predict, gold):
    """
    Classifies the prediction as correct or incorrect
    input: predict, gold - dictionaries, with keys as labels and values as constituents
    output: total missed, total correct, total incorrect 
    """
    correct = 0
    incorrect = 0
    missed = 0

    for key in gold.keys():
        if key in predict.keys():
            if gold[key].lower() == predict[key].lower():
                correct += 1
            else:
                incorrect += 1
        else:
            missed += 1
    if incorrect > 0:
        return 0
    
    elif missed > 0:
        return 0

    else:
        return 1

def get_description(verb, srl_output):
    """
    Returns the description of the verb
    """
    for verb1 in srl_output['verbs']:
        
        if verb1['verb'] == verb:
            return verb1['description']
        

    return verb1['verb'] # Not correct, but its a temporary fix

def map_list_to_constituent(constituent, output_group_9):
    """
    Maps the output of group 9 to the constituent list
    """
    output_dict = {}

    for i, element in enumerate(output_group_9[1]):
        if element != "_":
            classified_at, label  = output_group_9[0][i], output_group_9[1][i]
            for element1 in constituent:
                if classified_at in element1:
                    output_dict[label] = element1

    return output_dict

def predict_SRL_BERT(text, gold):
    
    predicted_outputs = MODEL_SRL_BERT.predict(sentence = text)
    description = get_description(gold, predicted_outputs)
    return get_constituent_list(description), parse_string(description)




def predict_SRL(text, gold):
        
    predicted_outputs = MODEL_SRL.predict(sentence = text)


    
    return parse_string(get_description(gold, predicted_outputs))


def predict_srl_group9(text, constituent    ):

    predicted_outputs = predict_total(text)

    return map_list_to_constituent(constituent, predicted_outputs)


In [10]:
def test_four():
    """
    Testing the SRL model on the test data for Cleft sentences
    """
    df = pd.read_json("testdata/test4.json")
    tests = df['tests']

    for test in tests:
        sentence = test['sentence']
        golden_labels = test['propbank_golden_labels']

        predicate = golden_labels['V']
        constituent, output_srl_bert = predict_SRL_BERT(sentence, predicate)

        output_srl = predict_SRL(sentence, predicate)
        output_group_9 = predict_srl_group9(sentence, constituent)

        print(output_srl_bert, golden_labels)

        print(classify(output_srl_bert, golden_labels))
        print(classify(output_srl, golden_labels))
        print(classify(output_group_9, golden_labels))


# test_four()

{'ARG0': 'the teacher', 'V': 'gave', 'ARG2': 'the students', 'ARG1': 'a quiz'} {'V': 'gave', 'ARG0': 'the teacher', 'ARG1': 'the students', 'ARG2': 'a quiz', 'ARGM_TMP': 'yesterday'}
0
0
0
{'ARG0': 'the cat', 'V': 'caught', 'ARG1': 'the mouse'} {'V': 'caught', 'ARG0': 'the cat', 'ARG1': 'the mouse', 'ARGM_TMP': 'last night'}
0
0
0
{'ARG0': 'the student', 'V': 'found', 'ARG1': 'the missing book'} {'V': 'found', 'ARG0': 'the student', 'ARG1': 'the missing book', 'ARGM_LOC': 'in the library'}
0
0
0
{'ARG1': 'the concept of morality', 'ARG0': 'the philosopher', 'V': 'discussed'} {'V': 'discussed', 'ARG0': 'the philosopher', 'ARG1': 'the concept of morality', 'ARGM_ADV': 'at length'}
0
0
0
{'ARG0': 'the dog', 'V': 'chased', 'ARG1': 'the cat'} {'V': 'chased', 'ARG0': 'the dog', 'ARG1': 'the cat', 'ARGM_LOC': 'in the park'}
0
0
0
{'ARG1': 'the cake', 'ARG0': 'the chef', 'V': 'baked'} {'V': 'baked', 'ARG0': 'the chef', 'ARG1': 'the cake'}
1
1
0
{'ARG1': 'the book', 'ARG0': 'the author', 'V': '

In [64]:
def test_one():
    """
    Testing SRL capability on handeling, statement -> multiple realizations
    """
    # TODO check if predicted is correct
    df = pd.read_json("testdata/test1.json")
    tests = df['tests']

    for test in tests[:]:
        original = test['proposition']
        realizations = test['realizations']
        for real in realizations[:2]:
            sentence = real['realization']
            gold_label = real['propbank_golden_labels']
          
            predicate_gold = gold_label['V']

            
            constituent, output_srl = predict_SRL_BERT(sentence, predicate_gold)
            output_srl_bert = predict_SRL(sentence, predicate_gold)

            output_group_9 = predict_srl_group9(sentence, constituent)
            


            print(classify(output_srl, gold_label))
            print(classify(output_srl_bert, gold_label))
            print(classify(output_group_9, gold_label))
            

# test_one()

In [65]:
def test_two():
        
    """
    Testing SRL capability on handeling, statement -> Question
    """
    # TODO check if predicted is correct
    df = pd.read_json("testdata/test2.json")
    tests = df['tests']
    total_srl, total_srl_bert, total_group_9 = 0, 0, 0
    for test in tests:

        statement = test['statement']
        question = test['question']

        golden_labels = test['propbank_golden_labels'][0]

        # print(golden_labels)
        statement_gold = golden_labels['Statement']
        question_gold = golden_labels['Question']

        statement_pred = statement_gold['V']
        question_pred = question_gold['V']

        # print(statement)

        constituent, output_statement_srl = predict_SRL_BERT(statement, statement_pred)
        output_statement_srl_bert = predict_SRL(statement, statement_pred)
        output_statement_group_9 = pred.predict_srl_group9(statement, constituent)


        constituent, output_question_srl = predict_SRL(question, question_pred)

        output_question_srl_bert = predict_SRL_BERT(question, question_pred)
        output_question_group_9 = pred.predict_srl_group9(question, constituent)
        # print(question)
        # print(classify(output_statement_srl_bert, statement_gold), classify(output_question_srl_bert, question_gold))
        # print(output_question_srl_bert, question_gold)
        SRL_score = classify(output_statement_srl, statement_gold) * classify(output_question_srl, question_gold)

        SRL_BERT_score = classify(output_statement_srl_bert, statement_gold) * classify(output_question_srl_bert, question_gold)

        SRL_group_9_score = classify(output_statement_group_9, statement_gold) * classify(output_question_group_9, question_gold)

        

        # print(SRL_score)
        # print(SRL_BERT_score)
        # print(SRL_group_9_score)


# test_two()

In [ ]:
def test_three():
    df = pd.read_json("testdata/test3.json")
    tests = df['tests']
    srl, bert, group9 = 0, 0, 0
    for test in tests:
        active = test['active']
        passive = test['passive']

        golden_labels = test['propbank_golden_labels'][0]

        active_gold = golden_labels['Active']
        passive_gold = golden_labels['Passive']

        active_predicate = active_gold['V']
        passive_predicate = passive_gold['V']



        constituent, output_active_srl = predict_SRL_BERT(active, active_predicate)
        output_active_srl_bert = predict_SRL(active, active_gold)
        output_active_group_9 = predict_srl_group9(active, constituent)


        constituent, output_passive_srl = predict_SRL_BERT(passive, passive_predicate)
        output_passive_srl_bert = predict_SRL(passive, passive_predicate)
        output_passive_group_9 = predict_srl_group9(passive, constituent)

        print(output_active_srl, output_passive_srl,)
        print(classify(output_active_srl, active_gold), classify(output_passive_srl, passive_gold), active_gold, passive_gold)
        # print(output_active_srl_bert,  "\n\n", active_gold)
        srl += (classify(output_active_srl, active_gold) * classify(output_passive_srl, passive_gold))
        bert += (classify(output_active_srl_bert, active_gold) * classify(output_passive_srl_bert, passive_gold))
        group9 += (classify(output_active_group_9, active_gold) * classify(output_passive_group_9, passive_gold))

    print(srl, bert)
# test_three()

In [ ]:

df = pd.read_json("testdata/test4.json")
tests = df['tests']

for test in tests:
    sentence = test['sentence']
    golden_labels = test['propbank_golden_labels']

    output_srl = predict_SRL(sentence)
    output_srl_bert = predict_SRL_BERT(sentence)
    output_group_9 = predict_srl_group9(sentence)

    print(golden_labels, "\n\n", output_srl_bert)
    print(classify(output_srl_bert, golden_labels))

In [ ]:

df = pd.read_json("testdata/test5.json")
tests = df['tests']

for test in tests:
    print(test)